## Imports

In [ ]:
import sys
import os
import json
import ast
from collections import Counter
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

sys.path.append(os.path.abspath(".."))

In [ ]:
from tfm.settings import settings
import matplotlib as plt
from tfm.llm.llm_vllm import VLLMLLM
from tfm.llm.llm_openai import OpenAILLM
from tfm.utils.utils import (
    generar_grupos_para_llamadas,
    asignacion_proporcional,
    procesar_grupo,
    clean_text,
    read_cie10_file
)
from tfm.prompt_engineering.data_generation_prompts import (
    system_prompt_openai,
    user_prompt_openai,
    system_prompt_llama,
    user_prompt_llama,
)

## Parámetros

Con meta-llama/Llama-3.2-3B-Instruct si se elige un CANTIDAD_A_GENERAR_POR_LLAMADA = 15, suele no generar la '}' del final.

Si se le deja la temperatura a 0, para un mismo input, generará siempre los mismos diagnósticos.

In [ ]:
# MODEL = 'openai'
# MODEL_NAME = "gpt-4o-mini"

MODEL = 'vllm'
# MODEL_NAME = "meta-llama/Llama-3.1-8B"
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
# MODEL_NAME = "merged_models/finetuned-llama-diagnosticos" # Modelo finetuneado


UMBRAL_MINIMO = 100 # Cantidad mínima de ejemplos por etiqueta que debe haber (se generarán hasta UMBRAL_MINIMO por cada etiqueta)
CANTIDAD_A_GENERAR_POR_LLAMADA = 5 # cuantos más se generen, menos llamadas habrá que hacer al LLM y por lo tanto, al estar haber más generaciones por llamada y mrnos llamadas, el resultado final será en principio más diverso (porque dentro de una misma llamada no debería hacer dos veces el mismo diagnóstico sintético). además Será más barato porque se harán menos llamadas al LLM, y se repetirán menos los ejemplos de input. Un número demasiado alto podría hacer que no quepa en la ventana de contexto del LLM, por lo que hay que tener cuidado con esto. Aunque la ventana suele ser bastante grande. También existe el riesgo de que "alucine" más y termine generando diagnósticos que no corresponden a la etiqueta, ya que tratará de seguir inventando diagnósticos distintos.
MAX_EJEMPLOS_INPUT = 5 # cantidad máxima de diganósticos de ejemplo que se le mandarán al LLM en cada llamada.Si se pone un número muy bajo, puede que el modelo no tenga suficiente contexto para generar diagnósticos coherentes. Si se pone un número muy alto, puede que las llamadas que se hagan al modelo sean poco variadas (por lo que se repetirán muchos diagnósticos) y además puede que no quepa en la ventana de contexto del modelo. Por lo tanto, hay que encontrar un equilibrio.
TEMPERATURE = 0.5 # Temperatura del modelo. A mayor temperatura, más aleatorio será el resultado, y por lo tanto los ejemplos generados serán más diversos. A menor temperatura, más repetitivo y menos diverso será el resultado, pero habrá menos "alucinaciones" y por lo tanto los diagnósticos generados serán más coherentes.
FREQUENCY_PENALTY = 0.0 # Penalización por frecuencia de palabras. A mayor penalización, menos repetitivo será el resultado, y por lo tanto los ejemplos generados serán más diversos. A menor penalización, más repetitivo y menos diverso será el resultado, pero habrá menos "alucinaciones" y por lo tanto los diagnósticos generados serán más coherentes. Valor entre -2.0 y 2.0.
PRESENCE_PENALTY = 0.0 # Penalización por presencia de palabras. A mayor penalización, menos repetitivo será el resultado, y por lo tanto los ejemplos generados serán más diversos. A menor penalización, más repetitivo y menos diverso será el resultado, pero habrá menos "alucinaciones" y por lo tanto los diagnósticos generados serán más coherentes. Valor entre -2.0 y 2.0.

MAX_RETRIES = 3 # Cantidad máxima de reintentos en caso de error al llamar al LLM. Si se supera este número, se descartará la llamada y se continuará con la siguiente. Esto es útil para evitar que se generen menos datos de los que se deberían generar por errores en la llamada al LLM.
MAX_WORKERS = 20  # Número máximo de ejecuciones paralelas

In [ ]:
if MODEL == 'openai':
    system_prompt = system_prompt_openai
    user_prompt = user_prompt_openai
    llm = OpenAILLM(model_name=MODEL_NAME)
elif MODEL == 'vllm':
    system_prompt = system_prompt_llama
    user_prompt = user_prompt_llama
    llm = VLLMLLM(model_name=MODEL_NAME)

## Leer Datos

In [ ]:
data = pd.read_csv("../data/diagnoses_df/ground_truth_df.csv")
data

In [ ]:
etiquetas_diagnosticos = read_cie10_file("../data/diagnoses_df/diagnosticos_tipos.csv")
etiquetas_diagnosticos

In [ ]:
data['Codigos_diagnosticos_list'] = data['Codigos_diagnosticos'].apply(ast.literal_eval)
data['Codigos_diagnosticos_list']

## Conteo de etiquetas a nivel individual

In [ ]:
all_labels = [label for sublist in data['Codigos_diagnosticos_list'] for label in sublist]
label_counts = Counter(all_labels)
label_counts_df = pd.DataFrame(label_counts.items(), columns=['Etiqueta', 'Frecuencia'])
label_counts_df.sort_values(by='Frecuencia', ascending=False, inplace=False)

In [ ]:
label_counts_df.plot(x='Etiqueta', y='Frecuencia', kind='bar', figsize=(18, 5))

In [ ]:
etiquetas_pobres_df = label_counts_df[label_counts_df['Frecuencia'] < UMBRAL_MINIMO]

ejemplos_faltantes = {
    row['Etiqueta']: UMBRAL_MINIMO - row['Frecuencia']
    for _, row in etiquetas_pobres_df.iterrows()
}
ejemplos_faltantes

### Comprobar qué códigos pueden aparecer "solos" y cuáles solo aparecen acompañados de otros

In [ ]:
all_codes = etiquetas_diagnosticos
apariciones = {codigo: {'solo': False, 'acompanado': False} for codigo in all_codes}

for lista in data['Codigos_diagnosticos_list']:
    es_solo = (len(lista) == 1)
    for codigo in lista:
        if es_solo:
            apariciones[codigo]['solo'] = True
        else:
            apariciones[codigo]['acompanado'] = True

# ódigos que pueden aparecer solos (al menos una vez en lista de 1)
codigos_pueden_solo = [c for c, f in apariciones.items() if f['solo']]

# Códigos que solo aparecen acompañados (nunca en lista de 1)
codigos_solo_acompanado = [c for c, f in apariciones.items() if f['acompanado'] and not f['solo']]

print("Códigos que pueden aparecer solos:", codigos_pueden_solo)
print("Códigos que solo aparecen acompañados:", codigos_solo_acompanado)

In [ ]:
ejemplos_faltantes_individuales = {
    codigo: ejemplos_faltantes[codigo]
    for codigo in codigos_pueden_solo
    if codigo in ejemplos_faltantes
}
ejemplos_faltantes_acompanados = {
    codigo: ejemplos_faltantes[codigo]
    for codigo in codigos_solo_acompanado
    if codigo in ejemplos_faltantes
}

In [ ]:
ejemplos_faltantes_individuales

In [ ]:
ejemplos_faltantes_acompanados

## Conteo de grupos de etiquetas

In [ ]:
# Agrupar por conjunto (orden no importa, así que usamos set y lo convertimos a tuple ordenado)
data['Diagnosticos_normalizados'] = data['Codigos_diagnosticos_list'].apply(lambda x: tuple(sorted(x)))
grupo_conjuntos = data.groupby('Diagnosticos_normalizados').size().reset_index(name='Frecuencia')

print("\nFrecuencia por conjunto de etiquetas:")
grupo_conjuntos

### De los que solo aparecen acompañados

In [ ]:
df_filtrado = grupo_conjuntos[grupo_conjuntos['Diagnosticos_normalizados'].apply(lambda tupla: any(codigo in tupla for codigo in codigos_solo_acompanado))]
df_filtrado

## Generación de datos

Para cada etiqueta presente en ejemplos_faltantes hay que generar tantos diagnósticos como se indica. Hay 2 casos en los que hay que generar casi todos a partir de únicamente 1 o 2 ejemplos deisponibles, lo que peude ser complejo (pues es posible que lo que se genere tenga poca diversidad, al partir todos de los mismos ejemplos de base).

La idea es, para cada etiqueta, generar ejemplos a partir de los disponibles. Haciendo varias llamadas al LLM, cada una con algunos de los ejemplos disponibles.

### Para aquellas etiquetas que solamente aparecen acompañadas por otras

In [ ]:
ejemplos_faltantes_acompanados

In [ ]:
diagnosticos_generados_completo_list = []
# Para cada etiqueta, hay que usar proporcionalmente los ejemplos disponibles para generar los ejemplos sintéticos.
for etiqueta, faltantes in ejemplos_faltantes_acompanados.items():
    print(f"\n\nEtiqueta: {etiqueta} - Faltantes: {faltantes}")
    df_acom = data[
            data['Codigos_diagnosticos_list']
                .apply(lambda lst: etiqueta in lst and len(lst) > 1)
        ].copy()
    unique_combinations = list(df_acom['Diagnosticos_normalizados'].unique())
    grupos_ejemplos = []
    for combo in unique_combinations:
        grupo = df_acom[df_acom['Diagnosticos_normalizados'] == combo]
        ejemplos = grupo['Descripcion_diagnosticos'].tolist()
        
        etiquetas = grupo['Codigos_diagnosticos'].iloc[0]  
        
        grupos_ejemplos.append({
            "etiquetas": etiquetas,
            "ejemplos": ejemplos
        })

    tamaños = [len(g['ejemplos']) for g in grupos_ejemplos]
    asign = asignacion_proporcional(tamaños, total_faltantes=faltantes)

    for ejemplos, falt in zip(grupos_ejemplos, asign):
        ejemplos_con_etiqueta = ejemplos['ejemplos']
        etiquetas = ejemplos['etiquetas']

        if len(ejemplos_con_etiqueta) == 0: # Este caso es cñuando solo hay ejemplos para la etiqueta acompañada de otras etiquetas, y no hay ejemplos únicos para esa etiqueta.
            print(f"\n\t\tGrupo de Etiquetas: {etiquetas} - No hay ejemplos disponibles.")
            
        else:
            nombres_etiquetas = [etiquetas_diagnosticos[etiqueta] for etiqueta in json.loads(etiquetas.replace("'", '"'))]
            print(f"\n\tGrupo de Etiquetas: {etiquetas} ({nombres_etiquetas}) - Disponibles: {len(ejemplos_con_etiqueta)} | Faltantes: {falt}")
            
            grupos = generar_grupos_para_llamadas(ejemplos_con_etiqueta, falt, max_ejemplos_input=MAX_EJEMPLOS_INPUT, generar_por_llamada=CANTIDAD_A_GENERAR_POR_LLAMADA)
            print(f"\tCantidad de grupos generada: {len(grupos)}")
            diagnosticos_generados_total = []

            with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                futures = [
                    executor.submit(procesar_grupo, grupo, etiquetas, nombres_etiquetas, CANTIDAD_A_GENERAR_POR_LLAMADA, system_prompt, user_prompt, llm, MAX_RETRIES, TEMPERATURE, FREQUENCY_PENALTY, PRESENCE_PENALTY)
                    for grupo in grupos
                ]

                for future in as_completed(futures):
                    try:
                        resultado = future.result()
                        diagnosticos_generados_total.extend(resultado)
                    except Exception as e:
                        print(f"Error procesando grupo en paralelo: {e}")
                
            print(f"\t - Diagnósticos generados final: {len(diagnosticos_generados_total)}")
            for diag in diagnosticos_generados_total:
                data_to_append = {
                    "Descripcion_diagnosticos": diag,
                    "Codigos_diagnosticos": etiquetas,
                    "Diagnosticos_estandar": nombres_etiquetas
                }
                diagnosticos_generados_completo_list.append(data_to_append)
        print()
generated_groups_df = pd.DataFrame(diagnosticos_generados_completo_list)
generated_groups_df

### Para aquellas etiquetas que aparecen individualmente (aunque puedan aparecer tambien acompañadas)

In [ ]:
ejemplos_faltantes_individuales

In [ ]:
diagnosticos_generados_completo_list = []
# Para cada etiqueta, hay que usar proporcionalmente los ejemplos disponibles para generar los ejemplos sintéticos.
for etiqueta, faltantes in ejemplos_faltantes_individuales.items():
    ejemplos_con_etiqueta = list(data[data['Codigos_diagnosticos_list'].apply(lambda x: 'F68.0' in x and len(x)==1)]['Descripcion_diagnosticos'].unique())
    if len(ejemplos_con_etiqueta) == 0: # Este caso es cuando solo hay ejemplos para la etiqueta acompañada de otras etiquetas, y no hay ejemplos únicos para esa etiqueta.
        print(f"\nEtiqueta: {etiqueta} - No hay ejemplos disponibles.")
        
    else:
        nombre_etiqueta = etiquetas_diagnosticos[etiqueta]
        print(f"\nEtiqueta: {etiqueta} ({nombre_etiqueta}) - Disponibles: {len(ejemplos_con_etiqueta)} | Faltantes: {faltantes}")
        
        # Filtrar los ejemplos que contienen la etiqueta actual
        grupos = generar_grupos_para_llamadas(ejemplos_con_etiqueta, faltantes, max_ejemplos_input=MAX_EJEMPLOS_INPUT, generar_por_llamada=CANTIDAD_A_GENERAR_POR_LLAMADA)
        print(f"Cantidad de grupos generada: {len(grupos)}")
        diagnosticos_generados_total = []
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            futures = [
                executor.submit(procesar_grupo, grupo, etiquetas, nombres_etiquetas, CANTIDAD_A_GENERAR_POR_LLAMADA, system_prompt, user_prompt, llm, MAX_RETRIES, TEMPERATURE, FREQUENCY_PENALTY, PRESENCE_PENALTY)
                for grupo in grupos
            ]

            for future in as_completed(futures):
                try:
                    resultado = future.result()
                    diagnosticos_generados_total.extend(resultado)
                except Exception as e:
                    print(f"Error procesando grupo en paralelo: {e}")
            
        print(f"\t - Diagnósticos generados final: {len(diagnosticos_generados_total)}")
        for diag in diagnosticos_generados_total:
            data_to_append = {
                "Descripcion_diagnosticos": diag,
                "Codigos_diagnosticos": f"['{etiqueta}']",
                "Diagnosticos_estandar": [nombre_etiqueta]
            }
            diagnosticos_generados_completo_list.append(data_to_append)
    print()
generated_singles_df = pd.DataFrame(diagnosticos_generados_completo_list)
generated_singles_df

## Dataset_final

In [ ]:
df_generados = pd.concat([generated_groups_df, generated_singles_df], ignore_index=True)
df_generados.insert(1, 'Descripcion_diagnosticos_limpio', df_generados['Descripcion_diagnosticos'].apply(clean_text).to_list())
df_generados['generated'] = True
df_generados

In [ ]:
data['generated'] = False
df_final = pd.concat([data.drop([col for col in data.columns if col not in df_generados.columns], axis=1), df_generados], ignore_index=True)
df_final

In [ ]:
df_final['Codigos_diagnosticos_list'] = df_final['Codigos_diagnosticos'].apply(ast.literal_eval)

all_labels = [label for sublist in df_final['Codigos_diagnosticos_list'] for label in sublist]
label_counts = Counter(all_labels)
label_counts_df = pd.DataFrame(label_counts.items(), columns=['Etiqueta', 'Frecuencia'])
label_counts_df.sort_values(by='Frecuencia', ascending=False, inplace=False)

In [ ]:
label_counts_df.plot(x='Etiqueta', y='Frecuencia', kind='bar', figsize=(18, 5))

In [ ]:
etiquetas_pobres_df = label_counts_df[label_counts_df['Frecuencia'] < UMBRAL_MINIMO]

ejemplos_faltantes = {
    row['Etiqueta']: UMBRAL_MINIMO - row['Frecuencia']
    for _, row in etiquetas_pobres_df.iterrows()
}
ejemplos_faltantes

In [ ]:
final_df_filename = (
    f"../data/generated/"
    f"MODEL_{MODEL}_"
    f"MODELNAME_{MODEL_NAME.replace('/', '-')}_" # Sustituir la barra diagonal por un guión para evitar problemas con el nombre del archivo
    f"UMBRALMIN_{UMBRAL_MINIMO}_"
    f"CANTXCALL_{CANTIDAD_A_GENERAR_POR_LLAMADA}_"
    f"MAXINP_{MAX_EJEMPLOS_INPUT}_"
    f"TEMP_{TEMPERATURE}_"
    f"FREQPEN_{FREQUENCY_PENALTY}_"
    f"PRESENCEPEN_{PRESENCE_PENALTY}.csv"
)
df_final.to_csv(final_df_filename, index=False)
final_df_filename

In [ ]:
params_dict = {
    'creation_timestamp': datetime.now().isoformat(),
    'Model': MODEL,
    'Model_name': MODEL_NAME,
    'Umbral_minimo': UMBRAL_MINIMO,
    'Cantidad_a_generar_por_llamada': CANTIDAD_A_GENERAR_POR_LLAMADA,
    'Max_ejemplos_input': MAX_EJEMPLOS_INPUT,
    'Temperature': TEMPERATURE,
    'Frequency_penalty': FREQUENCY_PENALTY,
    'Presence_penalty': PRESENCE_PENALTY
}
with open(final_df_filename.replace('.csv', '.json'), 'w', encoding='utf-8') as f:
    json.dump(params_dict, f, ensure_ascii=False, indent=4)